In [ ]:
import plotly.express as px
import pandas as pd
import ast
from zensols.mednlp import ApplicationFactory

In [ ]:
data_folder = "data/raw/"
d_icd = pd.read_csv(data_folder + 'D_ICD_DIAGNOSES.csv.gz', usecols=['ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE']) # ICD Code lookup
diagnoses = pd.read_csv(data_folder + 'DIAGNOSES_ICD.csv.gz', usecols=['HADM_ID', 'ICD9_CODE']) # Linkage between ICD codes and Note events
drg = pd.read_csv(data_folder + 'DRGCODES.csv.gz', usecols=['HADM_ID','DESCRIPTION']) # DRG Codes
note_events= pd.read_csv(data_folder + 'NOTEEVENTS.csv.gz', usecols=['HADM_ID','TEXT'], nrows=20000) # Number of rows = 2,083,180 without filter # We will handle this later

In [ ]:
# Data Sizes
print(f"D_ICD Shape: {d_icd.shape}")
print(f"DIAGNOSES Shape: {diagnoses.shape}")
print(f"DRG Shape: {drg.shape}")
# print(f"Note Events Shape: {note_events.shape}")

In [ ]:
# Schemas
print(f"D_ICD Schema: {d_icd.columns}")
print(f"DIAGNOSES Schema: {diagnoses.dtypes}")
print(f"DRG Schema: {drg.columns}")
print(f"Note Events Schema: {note_events.columns}")

In [ ]:
note_events['TEXT'] = note_events['TEXT'].apply(lambda x: "\"" + str(x) + "\"")
diagnoses = diagnoses.groupby(['HADM_ID']).agg(tuple).map(list).reset_index()
joined = note_events.join(diagnoses.set_index("HADM_ID"), on=['HADM_ID'], how='inner')
# joined = joined.join(drg.set_index('HADM_ID'), on='HADM_ID', how='inner').groupby(['HADM_ID','TEXT', 'ICD9_CODE']).agg(tuple).map(list).reset_index()
# joined = joined.join(d_icd.set_index('ICD9_CODE'), on='ICD9_CODE', how='inner')

In [ ]:
print(joined.columns)
print(joined.shape)
display(joined.head(20))

In [ ]:
display(joined.loc[joined['HADM_ID'] == 100195])
display(joined.loc[joined['HADM_ID'] == 100195]['TEXT'].values[0])
display(len(joined.loc[joined['HADM_ID'] == 100195]['TEXT'].values[0]))

In [ ]:
display(d_icd.loc[d_icd['ICD9_CODE'] == '1890'])

In [ ]:
diagnoses.dtypes

## Look at Note Format

In [ ]:
joined['TEXT'][1]

### Examine Code Distribution

In [ ]:
# Get Distribution of codes from Diagnoses table
def remove_evm_codes(df: pd.DataFrame) -> pd.DataFrame:
    # Drop E, V, and M codes
    df['ICD9_CODE'] = df['ICD9_CODE'].apply(lambda x: str(x))
    output = df[~df['ICD9_CODE'].str.startswith("E")]
    output = output[~output['ICD9_CODE'].str.startswith("V")]   
    output = output[~output['ICD9_CODE'].str.startswith("M")]
    return output

# DIAGNOSES Schema: Index(['HADM_ID', 'ICD9_CODE'], dtype='object')

df = diagnoses[diagnoses['ICD9_CODE'].notna()]

df = remove_evm_codes(df)

# Drop codes that start with E, V, and M
df = remove_evm_codes(df)

# Shorten codes to 3 digits (for now)
df['ICD9_CODE'] = df['ICD9_CODE'].str.slice(0, 3)

# Explode list
df['ICD9_CODE'] = df['ICD9_CODE'].explode('ICD9_CODE').reset_index(drop=True)
df = df.drop(columns=['HADM_ID'])

# Get count for each ecode
df = df.value_counts().reset_index()

display(df.head())


In [ ]:
# Plot ALL

fig = px.bar(df.nlargest(columns=['count'], n=500), x='ICD9_CODE', y='count',color='ICD9_CODE', title='Code Counts')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()


In [ ]:
# Plot ALL

fig = px.bar(df.nlargest(columns=['count'], n=200), x='ICD9_CODE', y='count',color='ICD9_CODE', title='Code Counts')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
# Plot Scoped to MAX

# Filter out codes out of scope
subset_df = pd.read_csv("./data/joined/dataset_single_001_088.csv.gz")
subset_df.drop(columns=['TEXT'], inplace=True)

# Explode list
subset_df['ICD9_CODE'] = subset_df['ICD9_CODE'].explode('ICD9_CODE').reset_index(drop=True)
display(subset_df.head(3))
subset_df.drop(columns=['HADM_ID'], inplace=True)

# Get count for each ecode
subset_df = subset_df.value_counts().reset_index()



fig = px.bar(subset_df, x='ICD9_CODE', y='count',color='ICD9_CODE', title='Code Counts')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

## The Long Tail Problem

In [ ]:
# The TOP X codes represent what percent of the data? (Total Dataset)
x=260

top_100 = df.nlargest(columns=['count'], n=x)
top_100_sum = top_100['count'].sum()
total_sum = df['count'].sum()

print(f"Top {x} Codes represent {top_100_sum/total_sum*100:.2f}% of the data")

## Test out MedNLP Package for Note Parsing

In [ ]:
# Sample
doc_parser = ApplicationFactory.get_doc_parser()
doc = doc_parser('John was diagnosed with kidney failure')

for tok in doc.tokens: 
    print(tok.norm, tok.pos_, tok.tag_, tok.cui_, tok.detected_name_)

print(doc.entities)

In [ ]:
# Sample from MIMIC-III dataset

#print(f"***\n{joined['TEXT'][1]} \n***")

doc_parser = ApplicationFactory.get_doc_parser()
doc = doc_parser(joined['TEXT'][1])

print("**Parsed:**")
# print(doc.entities)
new_note = set([])
for tok in doc.tokens:
    if tok.is_concept and tok.tuis_ in ['T184', 'T047', 'T046', 'T033', 'T037','T191']:
        '''
        Maybe add?:
        T060: Diagnostic Procedure
        T061 Therapeutic or Preventive Procedure
        T033: Finding
        T046: Pathologic Function
        '''
        print(tok, tok.detected_name_, tok.sub_names, tok.pref_name_, tok.tuis_, tok.tui_descs_)
        new_note.add(tok.detected_name_.replace("~"," "))
        new_note.add(tok.pref_name_.lower())

print(f"**New Note:**\n{new_note}")